In [1]:
# 📘 Linear Probe Evaluation on SimCLR Encoder

# 📌 Cell 1: Imports and Setup
import os
import tensorflow as tf
import numpy as np
import wandb
from wandb.integration.keras import WandbCallback
from tensorflow.keras import layers, models, callbacks

# Constants
NUM_CROPS = 9
NUM_DISEASES = 33
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE
INPUT_SHAPE = (224, 224, 3)



2025-03-24 20:02:24.680248: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 20:02:24.707585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742826744.730116   16062 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742826744.736998   16062 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 20:02:24.766165: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# 📌 Cell 2: W&B Init
wandb.init(
    project="plantvillage_simclr",
    name="linear_probe_on_simclr_RUN_5",
    config={
        "model": "SimCLR + Linear Probe",
        "batch_size": BATCH_SIZE,
        "epochs": 300,
        "optimizer": "Adam",
        "learning_rate": 1e-4
    }
)



wandb: Currently logged in as: gsrivatsa1398 (gsrivatsa1398-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:
# 📌 Cell 3: TFRecord Paths
split_path = "/home/srivatsa/Multiple_Crop_Disease_Detection/Dataset/PlantVillage_Structured/AUG_TFRecord/split"
train_tfrecord = os.path.join(split_path, "train.tfrecord")
val_tfrecord = os.path.join(split_path, "val.tfrecord")
test_tfrecord = os.path.join(split_path, "test.tfrecord")



In [4]:
# 📌 Cell 4: TFRecord Parsing
feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "crop": tf.io.FixedLenFeature([], tf.int64),
    "disease": tf.io.FixedLenFeature([], tf.int64)
}

def parse_tfrecord(example_proto):
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed["image"], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    crop = tf.one_hot(parsed["crop"], depth=NUM_CROPS)
    disease = tf.one_hot(parsed["disease"], depth=NUM_DISEASES)
    return image, {"crop_output": crop, "disease_output": disease}



In [5]:
# 📌 Cell 5: Load Datasets
def load_dataset(path):
    dataset = tf.data.TFRecordDataset(path)
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

train_dataset = load_dataset(train_tfrecord)
val_dataset = load_dataset(val_tfrecord)
test_dataset = load_dataset(test_tfrecord)



I0000 00:00:1742826754.738442   16062 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [6]:
# 📌 Cell 6: Load SimCLR Encoder
encoder_path = "/home/srivatsa/Multiple_Crop_Disease_Detection/Models/SimCLR_Pretrained/best_simCLR_Encoder.keras"
simclr_encoder = tf.keras.models.load_model(encoder_path, compile=False)
simclr_encoder.trainable = False  # Freeze encoder


In [7]:
# 📌 Cell 7: Build Linear Classifier on SimCLR Encoder
inputs = layers.Input(shape=INPUT_SHAPE)
features = simclr_encoder(inputs)

crop_output = layers.Dense(NUM_CROPS, activation="softmax", name="crop_output")(features)
disease_output = layers.Dense(NUM_DISEASES, activation="softmax", name="disease_output")(features)

linear_probe_model = models.Model(inputs=inputs, outputs=[crop_output, disease_output])


In [8]:
# 📌 Cell 8: Compile Model
linear_probe_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss={"crop_output": "categorical_crossentropy", "disease_output": "categorical_crossentropy"},
    loss_weights={"crop_output": 1.0, "disease_output": 2.0},
    metrics={"crop_output": "accuracy", "disease_output": "accuracy"}
)

In [9]:
# 📌 Cell 9: Callbacks

cb = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(
        "/home/srivatsa/Multiple_Crop_Disease_Detection/Models/SimCLR_Pretrained/best_LP_Model/best_simclr_linear_probe_model.keras",
        save_best_only=True,
        monitor="val_disease_output_accuracy",
        mode="max"
    ),
    WandbCallback(
        log_model=False,    # 👈 disable model saving to avoid deprecated save_format issue
        save_model=False,   # 👈 must be False
        save_graph=False    # 👈 also disable graph saving (optional)
    )
]

wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.


In [10]:
# 📌 Cell 10: Train Model
history = linear_probe_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=300,
    callbacks=cb
)

Epoch 1/300


2025-03-24 20:02:46.467340: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
I0000 00:00:1742826766.736661   16223 service.cc:148] XLA service 0x7f4b7c0022a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742826766.736719   16223 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-03-24 20:02:47.079521: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742826765.756417   16223 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-24 20:02:47.507143: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 16 bytes spill stores, 16 bytes spill loads

2025-03-24 20:02:48.072731: I external/l

    496/Unknown 50s 54ms/step - crop_output_accuracy: 0.4006 - crop_output_loss: 1.7749 - disease_output_accuracy: 0.2312 - disease_output_loss: 2.9759 - loss: 7.7267

2025-03-24 20:03:30.495013: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 16 bytes spill stores, 16 bytes spill loads

2025-03-24 20:03:30.561126: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 20 bytes spill stores, 20 bytes spill loads

2025-03-24 20:03:30.712315: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 168 bytes spill stores, 168 bytes spill loads

2025-03-24 20:03:30.801146: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 632 bytes spill stores, 540 bytes spill loads

2025-03-24 20:03:30.826437: I external/local_xla/xla/stream_executor/cuda/cu

    497/Unknown 64s 83ms/step - crop_output_accuracy: 0.4009 - crop_output_loss: 1.7743 - disease_output_accuracy: 0.2315 - disease_output_loss: 2.9749 - loss: 7.7241

...

497/497 ━━━━━━━━━━━━━━━━━━━━ 32s 64ms/step - crop_output_accuracy: 0.9499 - crop_output_loss: 0.1616 - disease_output_accuracy: 0.9456 - disease_output_loss: 0.1873 - loss: 0.5363 - val_crop_output_accuracy: 0.9477 - val_crop_output_loss: 0.1761 - val_disease_output_accuracy: 0.9292 - val_disease_output_loss: 0.2453 - val_loss: 0.6666
Epoch 298/300
475/497 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - crop_output_accuracy: 0.9484 - crop_output_loss: 0.1636 - disease_output_accuracy: 0.9464 - disease_output_loss: 0.1868 - loss: 0.5371 

2025-03-24 22:51:15.116653: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14747032845222501135
2025-03-24 22:51:15.116723: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 15539020123906665649
2025-03-24 22:51:15.116756: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9713265875010720821


497/497 ━━━━━━━━━━━━━━━━━━━━ 35s 70ms/step - crop_output_accuracy: 0.9485 - crop_output_loss: 0.1634 - disease_output_accuracy: 0.9465 - disease_output_loss: 0.1867 - loss: 0.5368 - val_crop_output_accuracy: 0.9476 - val_crop_output_loss: 0.1758 - val_disease_output_accuracy: 0.9292 - val_disease_output_loss: 0.2449 - val_loss: 0.6653
Epoch 299/300


2025-03-24 22:51:22.516800: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14747032845222501135
2025-03-24 22:51:22.516889: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 15539020123906665649


497/497 ━━━━━━━━━━━━━━━━━━━━ 34s 69ms/step - crop_output_accuracy: 0.9499 - crop_output_loss: 0.1600 - disease_output_accuracy: 0.9468 - disease_output_loss: 0.1844 - loss: 0.5287 - val_crop_output_accuracy: 0.9481 - val_crop_output_loss: 0.1757 - val_disease_output_accuracy: 0.9291 - val_disease_output_loss: 0.2447 - val_loss: 0.6651
Epoch 300/300
497/497 ━━━━━━━━━━━━━━━━━━━━ 35s 70ms/step - crop_output_accuracy: 0.9492 - crop_output_loss: 0.1608 - disease_output_accuracy: 0.9442 - disease_output_loss: 0.1848 - loss: 0.5304 - val_crop_output_accuracy: 0.9484 - val_crop_output_loss: 0.1757 - val_disease_output_accuracy: 0.9289 - val_disease_output_loss: 0.2444 - val_loss: 0.6644


In [11]:
# 📌 Cell 11: Evaluate Model
test_metrics = linear_probe_model.evaluate(test_dataset)
print("\n✅ Final Test Evaluation:", test_metrics)

     70/Unknown 4s 49ms/step - crop_output_accuracy: 0.9385 - crop_output_loss: 0.1965 - disease_output_accuracy: 0.9254 - disease_output_loss: 0.2569 - loss: 0.7104 

2025-03-24 22:52:37.705134: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 20 bytes spill stores, 20 bytes spill loads

2025-03-24 22:52:37.959136: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 16 bytes spill stores, 16 bytes spill loads

2025-03-24 22:52:38.049617: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 168 bytes spill stores, 168 bytes spill loads

2025-03-24 22:52:38.115034: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 632 bytes spill stores, 540 bytes spill loads



71/71 ━━━━━━━━━━━━━━━━━━━━ 11s 153ms/step - crop_output_accuracy: 0.9385 - crop_output_loss: 0.1962 - disease_output_accuracy: 0.9254 - disease_output_loss: 0.2570 - loss: 0.7103

✅ Final Test Evaluation: [0.7080945372581482, 0.18611587584018707, 0.26098448038101196, 0.9401013255119324, 0.9249064326286316]


In [12]:
# 📌 Cell 12: Save Final Model
save_path = "/home/srivatsa/Multiple_Crop_Disease_Detection/Models/SimCLR_Pretrained/not_best_simclr_linear_probe_model.keras"
linear_probe_model.save(save_path)
print("✅ Linear Probe Model Saved")

✅ Linear Probe Model Saved
